In [1]:
import torch
import faiss
import numpy as np
import torchvision
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.decomposition import PCA
from torchvision import transforms as T
from torchvision.models.vgg import vgg19
from torchvision.models.resnet import resnet50


img = Image.open('../data/test-elephant.png')
print(img)

<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1600x1062 at 0x12A2146D0>


## Load models

In [5]:
vgg_model = vgg19()
resnet_model = resnet50()

## Test DinoV2

In [27]:
dinov2 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')

transform = T.Compose([
    T.Resize(520),
    T.CenterCrop(518),           
    T.ToTensor(),               
    T.Normalize(mean=0.5, std=0.2)
])

patch_size = dinov2.patch_size # patchsize=14

patch_h  = 520//patch_size
patch_w  = 520//patch_size

feat_dim = 1024

img = transform(img)

with torch.no_grad():
    features_dict = dinov2.forward_features(img.unsqueeze(0))
    features = features_dict['x_norm_patchtokens']

# pca = PCA(n_components=3)
# pca.fit(features)

# pca_features = pca.transform(features)
# pca_features = (pca_features - pca_features.min()) / (pca_features.max() - pca_features.min())
# pca_features = pca_features * 255

# plt.imshow(pca_features.reshape(16, 16, 3).astype(np.uint8))
# plt.savefig('meta_dog_features.png')

# In dinov2/models/vision_transformer.py line 290 add

# def forward(self, *args, is_training=False, return_patches=False, **kwargs):
# ret = self.forward_features(*args, **kwargs)
# if is_training:
# return ret
# elif return_patches:
# return ret["x_norm_patchtokens"]
# else:
# return self.head(ret["x_norm_clstoken"])

Using cache found in /Users/michal/.cache/torch/hub/facebookresearch_dinov2_main


NotImplementedError: No operator found for `memory_efficient_attention_forward` with inputs:
     query       : shape=(1, 1370, 16, 64) (torch.float32)
     key         : shape=(1, 1370, 16, 64) (torch.float32)
     value       : shape=(1, 1370, 16, 64) (torch.float32)
     attn_bias   : <class 'NoneType'>
     p           : 0.0
`decoderF` is not supported because:
    device=cpu (supported: {'cuda'})
    attn_bias type is <class 'NoneType'>
`flshattF@0.0.0` is not supported because:
    device=cpu (supported: {'cuda'})
    dtype=torch.float32 (supported: {torch.float16, torch.bfloat16})
    operator wasn't built - see `python -m xformers.info` for more info
`tritonflashattF` is not supported because:
    device=cpu (supported: {'cuda'})
    dtype=torch.float32 (supported: {torch.float16, torch.bfloat16})
    operator wasn't built - see `python -m xformers.info` for more info
    triton is not available
`cutlassF` is not supported because:
    device=cpu (supported: {'cuda'})
`smallkF`

## Test FAISS